# Data Preprocessing for Water Analysis in Mexico City

* Date : Aug 2025
* Author : Axel Daniel Malváez Flores
* Description : This notebook contains the data preprocessing steps for the water analysis project in Mexico City. It includes loading, cleaning, and transforming various datasets related to water consumption, property data, and population data.

### Libraries

In [246]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pyarrow as pa
import pyarrow.dataset as ds

from unidecode import unidecode

import json
import warnings

warnings.filterwarnings("ignore")

### Constants

In [247]:
# WATER DATASETS

# Data consumption during the whole 2019 by neighborhood
watConsPath="../raw-data/water/consumo/consumo_agua_historico_2019.csv"
# Municipalities with drought data and higher/lower probability of drought
droughtPath="../raw-data/water/sequia/Municipios_con_ sequia.xlsx"
# Water Reports in Mexico City by neighborhood.
reports2224Path="../raw-data/water/reportes/reportes_agua_2024_01.csv"
reportsHistory="../raw-data/water/reportes/reportes_agua_hist.csv"
# Hidric feasibility
hidFeasPath="../raw-data/water/factibilidad/factibilidad-hdrica.json"
# Consumption every two months by neighborhood
habPath=("../raw-data/water/consumo/consumo-habitacional-promedio-"
           "bimestral-de-agua-por-colonia-m3.json")    

# PROPERTY DATASETS

# Index SHF for housing price in the area
indexSHFPath="../raw-data/property-population/SHF/indice_SHF.csv"
# Population Growth Rate
growthRateAlcPath=("../raw-data/property-population/CrecimientoPoblacional/"
                     "poblacion_total_tasa_crecimiento_alcaldia_1.2.csv")
# Density
housesColPath=("../raw-data/property-population/"
                "Hogares por colonia/hogares_colonia.shp")
# Concentración habitacional
densPath = ("../raw-data/property-population/alta_concentracion/"
            "zonas_vivienda.shp")

# ALCALDIA DELIMITATION

alcaldiaPath = "../raw-data/water/sequia/limite-de-las-alcaldias.json"

### Data Reading and Preprocessing

In [248]:
# ---------------------
# WATER DATA
# ---------------------

# Water Consumption 2019
watCons19 = pd.read_csv(watConsPath)

# Drought MEX CITY
drought = pd.read_excel(droughtPath, sheet_name='MONITOR - SEMAFORO - USO EFIC')

# Water Reports in Mexico City by neighborhood.
reports2224 = pd.read_csv(reports2224Path)
reportsHist = pd.read_csv(reportsHistory)

# Hidric feasibility
with open(hidFeasPath, "r", encoding="utf-8") as f : 
    feasFeat = json.load(f)
feasibility = gpd.GeoDataFrame.from_features(feasFeat["features"])

# Consumption every two months by neighborhood
with open(habPath, "r", encoding="utf-8") as f : 
    habConsFeats = json.load(f)
habCons = gpd.GeoDataFrame.from_features(habConsFeats['features'])

# ---------------------
# PROPERTY DATA
# ---------------------

hogaresCol = gpd.read_file(housesColPath) 
indexSHF = pd.read_csv(indexSHFPath, encoding='iso-8859-1', delimiter=';')
popGrowthRateAlc = pd.read_csv(growthRateAlcPath,encoding='utf-8')
density = gpd.read_file(densPath)


# --------------------
# ALCALDIA DATA
# --------------------

with open(alcaldiaPath, 'r', encoding='utf-8') as f:
    data = json.load(f)
alcaldiaGeo = gpd.GeoDataFrame.from_features(data['features'])

### Processing Datasets

#### Water Consumption Data 2019

Información bimestral por el concepto de suministro de agua a nivel manzana, considerando la facturación por servicio de consumo medido y promedio.

In [249]:
watCons19.fillna(0, inplace=True)

In [250]:
watCons19.drop_duplicates(inplace=True)

# Date 
watCons19['fecha_referencia'] = pd.to_datetime(watCons19['fecha_referencia'],
                                               format='ISO8601')

# Drop Duplicates 
watCons19[['fecha_referencia', 'anio', 'bimestre', 'indice_des', 'colonia',
           'alcaldia', 'latitud', 'longitud']].drop_duplicates(inplace=True)

# Adding derived columns for total consumption
inDom = watCons19['consumo_total_dom'] / watCons19['consumo_prom_dom']
watCons19['inmuebles_domesticos'] = inDom

inNoDom = watCons19['consumo_total_no_dom'] / watCons19['consumo_prom_no_dom']
watCons19['inmuebles_no_domesticos'] = inNoDom

inMix = watCons19['consumo_total_mixto'] / watCons19['consumo_prom_mixto']
watCons19['inmuebles_mixtos'] = inMix

# replace NaN values with 0 in the derived column
watCons19['inmuebles_domesticos'].fillna(0, inplace=True)
watCons19['inmuebles_no_domesticos'].fillna(0, inplace=True)
watCons19['inmuebles_mixtos'].fillna(0, inplace=True)

# Data structure and column ordering
watCons19 = watCons19[['fecha_referencia', 'consumo_total',
                       'inmuebles_domesticos', 'consumo_total_dom',
                       'consumo_prom_dom', 'inmuebles_no_domesticos',
                       'consumo_total_no_dom', 'consumo_prom_no_dom',
                       'inmuebles_mixtos', 'consumo_total_mixto',
                       'consumo_prom_mixto', 'indice_des', 'colonia',
                       'alcaldia']]

# Adding total consumption and total number of properties
watCons19['total_inmuebles'] = watCons19['inmuebles_domesticos'] + watCons19['inmuebles_no_domesticos'] + watCons19['inmuebles_mixtos'] 

* OUTPUT

In [251]:
def normalizer(df, col, type='lower'):
    if type == 'lower':
        df[col] = df[col].apply(lambda x : str(x).lower())
    else :
        df[col] = df[col].apply(lambda x : str(x).title())

    df[col] = df[col].apply(lambda x : unidecode(x))

In [252]:
normalizer(watCons19, "colonia", "lower")
normalizer(watCons19, "alcaldia", "title")

watCons19.to_csv("../data/consumoAgua19.csv", index=False)

#### Habitational Consumption Data

Diccionario : 

**SUM_cons_t**   
Suma del consumo total de agua por colonia (m3) por bimestre

**MEAN_cons_**   
Promedio de consumo total de colonia (m3) por bimestre

**VIV2010**   
Vivienda censo 2010

**PROMVIVCON**   
"Promedio de consumo total de agua de la colonia por número de viviendas"

**C_PROMVIVC**   
"Campo reclasificado en cuantiles en 5 rangos a partir de "PROMVIVCON"; donde el valor "5" corresponde a un consumo de agua muy alto, mientras que el número "1" a un consumo bajo"



In [253]:
normalizer(habCons, "colonia", "lower")
normalizer(habCons, "alcaldia", "title")

habCons = habCons.drop(columns=["cve_ent", "OBJECTID", "FREQUENCY"])

In [254]:
# Last Value is a None value
habCons = habCons.iloc[:-1]

#### Habitational Density

In [255]:
# 1. Both GeoDataFrames use the same coordinate reference system (CRS)
hogaresCol = hogaresCol.to_crs(density.crs)

In [256]:
# 2. Perform spatial join
hogares_with_grado = gpd.sjoin(
    hogaresCol,
    density[['geometry', 'grado', 'ID']],  # only keep necessary columns
    predicate='intersects',  # 'intersects' if geometries overlap partially
    how='left'
)

In [257]:
hogares_with_grado = hogares_with_grado.sort_values(by="ID", ascending=False).drop_duplicates(subset="cve_col").sort_values(by="cve_col")

In [258]:
normalizer(hogares_with_grado, "colonia", "lower")
normalizer(hogares_with_grado, "alcaldia", "title")

#### Saving both datasets

In [259]:
# Last Value is a None value
habCons = pd.merge(habCons,
                   hogares_with_grado[['cve_col', 'Sum_TotHog']],
                   on="cve_col",
                   how="left")

habCons = habCons.sort_values(by="SUM_cons_t", ascending=False).reset_index()

In [260]:
habCons.to_csv("../data/habCons.csv", index=False)
hogares_with_grado.to_csv("../data/hogaresColDensity.csv", index=False)

#### Factibility Data

| Grado de Factibilidad Hídrica | Descripción / Significado                                | Color       
| ----------------------------- | -------------------------------------------------------- | ----------- 
| Alta factibilidad 🌟          | Zonas con alta capacidad para mitigar riesgo de escasez  | 🟩 Verde    
| Media-alta factibilidad ⚠️    | Zonas con buena capacidad, pero con algunas limitaciones | 🟨 Amarillo 
| Media-baja factibilidad 🔶    | Zonas con capacidad limitada para mitigar riesgo         | 🟧 Naranja  
| Baja factibilidad ❌           | Zonas con poca o nula capacidad para mitigar riesgo      | 🟥 Rojo    

In [261]:
# Deleting last value since it contains an invalid coordinate
feasibility = feasibility.iloc[:-1,:]
feasibility.rename(columns={'NOMBRE':'colonia', 'DELEGACIO' : 'alcaldia'}, inplace=True)

In [262]:
normalizer(feasibility, "colonia", "lower")
normalizer(feasibility, "alcaldia", "title")

In [263]:
factibilidadNorm = gpd.sjoin(
    feasibility,
    habCons[['geometry', 'colonia', 'alcaldia']],  # only keep necessary columns
    predicate='intersects',  # 'intersects' if geometries overlap partially
    how='left'
)

factibilidadNorm = factibilidadNorm.reset_index()

In [264]:
def jaccard_similarity_words(text1, text2):
    """
    Compute Jaccard similarity between two texts
    using words as tokens.
    """
    # Convert to lowercase and split into words
    set1 = set(text1.lower().split())
    set2 = set(text2.lower().split())
    
    # Compute intersection and union
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    if union == 0:
        return 0
    return intersection / union

In [265]:
idxList = []
for idx, row in factibilidadNorm.iterrows():
    simCol = jaccard_similarity_words(row["colonia_left"], row["colonia_right"])
    simAlc = jaccard_similarity_words(row["alcaldia_left"], row["alcaldia_right"])
    if 0.3 < simCol and simAlc == 1:
        idxList.append(idx)        

len(idxList)

2408

In [266]:
factibilidadNorm = factibilidadNorm.iloc[idxList, ]

factibilidadNorm.drop_duplicates(subset=["colonia_right", "alcaldia_right"], inplace=True)
factibilidadNorm.drop_duplicates(subset=["colonia_left", "alcaldia_left"], inplace=True)

factibilidadNorm = factibilidadNorm.drop(columns=["colonia_left", "alcaldia_left", "index_right", "ID", "ENTIDAD", "MUNICIPIO"])

factibilidadNorm.rename(columns={"colonia_right" : "colonia",
                                 "alcaldia_right" : "alcaldia"}, inplace = True)

In [267]:
factibilidadNorm.to_csv("../data/feasibilityMexCity.csv", index=False)

#### Probabilidad de Escasez

In [268]:
cldCols = drought.columns.to_series().where(
    ~drought.columns.str.contains('^Unnamed'), '')
drought.columns = [x + '/' + y if x != '' else y 
                    for x, y in zip(cldCols, drought.iloc[0].astype(str))]
drought = drought.iloc[1:].reset_index().iloc[:,1:]

In [269]:
# Setting CRS to standard and geo interface
alcaldiaCDMX = alcaldiaGeo.set_crs(epsg=4326, inplace=True)
geojson = alcaldiaCDMX.__geo_interface__

# Identify idVars (the static columns)
idVars = ['Region de lluvias y fecha de solicitud de corroboración', 
           'CVE_CONCATENADA', 'CVE_ENT', 'CVE_MUN', 'NOMBRE_MUN', 'ENTIDAD',
           'ORG_CUENCA', 'CLV_OC', 'CON_CUENCA', 'CVE_CONC']

# Melt all other columns
meltedDF = drought.melt(id_vars=idVars,
                         var_name='variable',
                         value_name='value')

# Create category and date columns
category = []
date = []
for i in meltedDF['variable'].astype(str):
    tup = i.split('/')
    category.append(tup[0])
    date.append(tup[1])

meltedDF['CATEGORY'] = category
meltedDF['CATEGORY'] = meltedDF['CATEGORY'].str.replace(r'\.\d+$', '',
                                                          regex=True)

meltedDF['DATE'] = pd.to_datetime(date)
meltedDF['MONTH'] = meltedDF['DATE'].dt.month_name()
meltedDF['YEAR'] = meltedDF['DATE'].dt.year
meltedDF['DATE'] = meltedDF['DATE'].dt.strftime('%Y-%m')

# Mexico city filter, adding map and taking MAGNITUD value only
mxcDrought = meltedDF[(meltedDF['ENTIDAD'] == 'Ciudad de México')
                        & (meltedDF['CATEGORY'] == 'MAGNITUD')]
mxcDrought = pd.merge(mxcDrought, alcaldiaCDMX[['CVE_MUN', 'geometry']],
                       on='CVE_MUN', how='left')

color_map = {
    'SIN SEQUIA': '#f0f0f0',     # Very light gray / near white
    'PRE-ALERTA': '#a6d96a',     # Light gray
    'VERDE': '#1a9850',          # Dark green
    'AMARILLO': '#ffffbf',       # Yellow
    'NARANJA': '#fdae61',        # Orange
    'ROJO': '#d73027'            # Red
}

mxcDrought['color'] = mxcDrought[mxcDrought['CATEGORY'] == 'MAGNITUD']['value'].map(color_map)
mxcDrought = gpd.GeoDataFrame(mxcDrought, geometry='geometry')
# mxcDrought["geometry"] = mxcDrought["geometry"].simplify(tolerance=0.0005,
# preserve_topology=True)

# Data Processing
number_map = {
    'SIN SEQUIA': 1,     # Very light gray / near white
    'PRE-ALERTA': 2,     # Light gray
    'VERDE': 3,          # Dark green
    'AMARILLO': 4,       # Yellow
    'NARANJA': 5,        # Orange
    'ROJO': 6            # Red
}

mxcDrought['VALUE'] = mxcDrought[mxcDrought['CATEGORY'] == 'MAGNITUD']['value'].map(number_map)
mxcDrought['DATE'] = pd.to_datetime(mxcDrought['DATE'])

In [270]:
mappingValue = {
    6 : "Sequía crítica",
    5 : "Sequía severa",
    4 : "Sequía moderada",
    3 : "Sequía mínima",
    2 : "Anormalmente seco",
    1 : "Sin sequía"   
}

mxcDrought["DESC"] = mxcDrought["VALUE"].map(mappingValue)

* OUTPUT

In [271]:
normalizer(mxcDrought, "NOMBRE_MUN", "title")

In [272]:
mxcDroughtDropped = mxcDrought.drop(columns=['Region de lluvias y fecha de solicitud de corroboración', 'CVE_CONCATENADA', 'CVE_ENT', 'CLV_OC', 'variable', 'ORG_CUENCA', 'CON_CUENCA', 'CVE_CONC', 'CATEGORY'])

In [273]:
mxcDroughtDropped.to_csv("../data/droughtMexCity.csv", index=False)

#### Reportes de Agua

Info : 
- Data ranging from 2018 to 2024

* History of reports
    * The most common report
    * Separate all this by alcaldia and colonia

* Analyze time and identify trends/patterns in which all this reports are made.
    * Tipically after what hour or during which range of hours.

In [274]:
reports2224 = reports2224[['fecha_reporte','hora_reporte', 'reporte',
                           'alcaldia_catalogo', 'colonia_catalogo',
                           'longitud', 'latitud']]
reportsHist = reportsHist[['fecha','tipo_de_falla', 'alcaldia',
                           'colonia_datos_abiertos', 'longitud', 'latitud']]

new_names_cols = {'tipo_de_falla' : 'reporte', 
                  'alcaldia_catalogo' : 'alcaldia', 
                  'colonia_catalogo' : 'colonia',
                  'colonia_datos_abiertos' : 'colonia',
                  'fecha_reporte' : 'fecha'
                  }

reportsHist.rename(columns=new_names_cols, inplace=True)
reports2224.rename(columns=new_names_cols, inplace=True)

all_reports = pd.concat([reports2224, reportsHist], axis=0)

all_reports['fecha'] = pd.to_datetime(all_reports['fecha'])
all_reports['year'] = all_reports['fecha'].dt.year
all_reports['month_name'] = all_reports['fecha'].dt.month_name()
all_reports['month'] = all_reports['fecha'].dt.month

all_reports = all_reports.dropna(subset=['alcaldia', 'colonia'])

* OUTPUT

In [275]:
normalizer(all_reports, "colonia", "lower")
normalizer(all_reports, "alcaldia", "title")

In [276]:
all_reportsDropped = all_reports.drop(columns=["hora_reporte"])

In [277]:
all_reportsDropped.to_csv("../data/reportsAllHist.csv", index=False)

#### SHF Index

In [278]:
indexSHF['Indice'] = indexSHF['Indice'].replace(r",", ".",regex=True)
indexSHF['Indice'] = indexSHF['Indice'].astype(float)

dict_trimestres = {1:'03',
                   2:'06',
                   3:'09',
                   4:'12'}


indexSHF['fecha'] = indexSHF.apply(lambda x : str(x['Año']) + '-' + dict_trimestres[int(x['Trimestre'])], axis = 1)
indexSHF['fecha'] = pd.to_datetime(indexSHF['fecha'])

* OUTPUT

In [279]:
normalizer(indexSHF, "Municipio", "title")

In [280]:
indexSHF = indexSHF.drop(columns=["Consecutivo"])

In [281]:
indexSHF.to_csv("../data/indexSHF.csv", index=False)

## Preparando los datos

* Water Consumption 2019

In [282]:
# Escribe particionado (rápido y barato de leer)
table = pa.Table.from_pandas(watCons19, preserve_index=False)
ds.write_dataset(
    table,
    base_dir="../parquets/consumo19",       # genera carpetas
    format="parquet",
    #partitioning=["fecha_referencia","alcaldia"], # crea year=2024/alcaldia=Benito Juarez/...
    partitioning_flavor="hive",
    existing_data_behavior="overwrite_or_ignore",
)

* Factibilidad

In [295]:
factibilidadNorm['geometry'] = factibilidadNorm['geometry'].astype(str)

In [296]:
# Escribe particionado (rápido y barato de leer)
table = pa.Table.from_pandas(factibilidadNorm, preserve_index=False)
ds.write_dataset(
    table,
    base_dir="../parquets/factibilidad",       # genera carpetas
    format="parquet",
    #partitioning=["alcaldia","fact_hidr"], # crea year=2024/alcaldia=Benito Juarez/...
    partitioning_flavor="hive",
    existing_data_behavior="overwrite_or_ignore",
)

* Consumo Habitacional

In [285]:
habCons['geometry'] = habCons['geometry'].astype(str)

In [286]:
# Escribe particionado (rápido y barato de leer)
table = pa.Table.from_pandas(habCons, preserve_index=False)
ds.write_dataset(
    table,
    base_dir="../parquets/habCons",       # genera carpetas
    format="parquet",
    #partitioning=["alcaldia"], # crea year=2024/alcaldia=Benito Juarez/...
    partitioning_flavor="hive",
    existing_data_behavior="overwrite_or_ignore",
)

* Probabilidad de Escasez

In [287]:
mxcDroughtDropped['geometry'] = mxcDroughtDropped['geometry'].astype(str)

In [288]:
# Escribe particionado (rápido y barato de leer)
table = pa.Table.from_pandas(mxcDroughtDropped, preserve_index=False)
ds.write_dataset(
    table,
    base_dir="../parquets/drought",       # genera carpetas
    format="parquet",
    #partitioning=["YEAR"], # crea year=2024/alcaldia=Benito Juarez/...
    partitioning_flavor="hive",
    existing_data_behavior="overwrite_or_ignore",
)

* Reportes de Agua

In [289]:
# Escribe particionado (rápido y barato de leer)
table = pa.Table.from_pandas(all_reportsDropped, preserve_index=False)
ds.write_dataset(
    table,
    base_dir="../parquets/reportes",       # genera carpetas
    format="parquet",
    #partitioning=["reporte"], # crea year=2024/alcaldia=Benito Juarez/...
    partitioning_flavor="hive",
    existing_data_behavior="overwrite_or_ignore",
)

* Hogares Densidad

In [290]:
hogares_with_grado['geometry'] = hogares_with_grado['geometry'].astype(str)

In [291]:
# Escribe particionado (rápido y barato de leer)
table = pa.Table.from_pandas(hogares_with_grado, preserve_index=False)
ds.write_dataset(
    table,
    base_dir="../parquets/densidadHogares",       # genera carpetas
    format="parquet",
    #partitioning=["alcaldia"], # crea year=2024/alcaldia=Benito Juarez/...
    partitioning_flavor="hive",
    existing_data_behavior="overwrite_or_ignore",
)

* Index SHF

In [292]:
# Escribe particionado (rápido y barato de leer)
table = pa.Table.from_pandas(indexSHF, preserve_index=False)
ds.write_dataset(
    table,
    base_dir="../parquets/indexSHF",       # genera carpetas
    format="parquet",
    #partitioning=["Año"], # crea year=2024/alcaldia=Benito Juarez/...
    partitioning_flavor="hive",
    existing_data_behavior="overwrite_or_ignore",
)